##Тема «POS-tagger и NER»

Задание. Написать теггер на данных с русским языком
* проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
* написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
* сравнить все реализованные методы, сделать выводы
 



In [ ]:
pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import nltk
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger
import pyconll
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('tagsets')

[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [ ]:
RANDOM_STATE = 42

In [ ]:
!wget ./dataset_ru/ru_syntagrus-ud-train-a.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget ./dataset_ru/ru_syntagrus-ud-train-b.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget ./dataset_ru/ru_syntagrus-ud-train-c.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
!wget ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-12-14 09:31:45--  http://./dataset_ru/ru_syntagrus-ud-train-a.conllu
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
--2022-12-14 09:31:45--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train-a.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   157MB/s    in 0.2s    

2022-12-14 09:31:48 (157 MB/s) - ‘ru_syntagrus-ud-train-a.conllu’ saved [40736581/40736581]

FINISHED --2022-12-14 09:31:48--
Total wall clock time: 3.2s
Downloaded: 1 files, 39M in 0.2s (157 MB/s)
--2022-12-14 09:31:48--  http://./dataset_ru/ru

Загрузка данных

In [ ]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
full_train_b = pyconll.load_from_file('ru_syntagrus-ud-train-b.conllu')
full_train_c = pyconll.load_from_file('ru_syntagrus-ud-train-c.conllu')

full_train.extend([*full_train_b, *full_train_c]) # объединяем

full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [ ]:
fdata_train = []
for sent in full_train:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test:
    fdata_test.append([(token.form, token.upos) for token in sent])

fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [ ]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len("" if token.form is None else token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [ ]:
all_train_texts = [' '.join("" if token.form is None else token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join("" if token.form is None else token.form for token in sent) for sent in full_test]

all_train_labels = [' '.join("" if token.upos is None else token.upos for token in sent) for sent in full_train]
all_test_labels = [' '.join("" if token.upos is None else token.upos for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

Тест различных тэггеров и их комбинации.
Внесём получившиеся значения accuracy в таблицу:

In [ ]:
%%time

default_tagger = DefaultTagger('NOUN') #NOUN
possible_options = {
    'Trigram': 'TrigramTagger(fdata_train)',
    'Bigram': 'BigramTagger(fdata_train)',
    'Unigram': 'UnigramTagger(fdata_train)',
    'Default': 'default_tagger',
    'Trigram + Default': 'TrigramTagger(fdata_train, backoff=default_tagger)',
    'Trigram + Bigram': 'TrigramTagger(fdata_train, backoff=BigramTagger(fdata_train))',
    'Trigram + Bigram + Default': 'TrigramTagger(fdata_train, backoff=BigramTagger(fdata_train, backoff=default_tagger))',
    'Trigram + Unigram': 'TrigramTagger(fdata_train, backoff=UnigramTagger(fdata_train))',
    'Trigram + Unigram + Default': 'TrigramTagger(fdata_train, backoff=UnigramTagger(fdata_train, backoff=default_tagger))',
    'Trigram + Bigram + Unigram': 'TrigramTagger(fdata_train, backoff=BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train)))',
    'Trigram + Bigram + Unigram + Default': 'TrigramTagger(fdata_train, backoff=BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train, backoff=default_tagger)))',
    'Bigram + Default': 'BigramTagger(fdata_train, backoff=default_tagger)',
    'Bigram + Unigram': 'BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train))',
    'Bigram + Unigram + Default': 'BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train, backoff=default_tagger))',
    'Unigram + Default': 'UnigramTagger(fdata_train, backoff=default_tagger)'    
}

scores = []

for name, tagger in possible_options.items():
    scores.append((round(eval(tagger).accuracy(fdata_test), 3)))

CPU times: user 2min 32s, sys: 427 ms, total: 2min 33s
Wall time: 2min 33s


In [ ]:
comparison = pd.DataFrame(scores, index=possible_options.keys(), columns=['Score']).sort_values('Score', ascending=False)
comparison

,Score
Trigram + Unigram + Default,0.912
Trigram + Bigram + Unigram + Default,0.912
Bigram + Unigram + Default,0.912
Unigram + Default,0.906
Bigram + Unigram,0.884
Trigram + Unigram,0.883
Trigram + Bigram + Unigram,0.883
Unigram,0.878
Trigram + Bigram + Default,0.861
Bigram + Default,0.861


Лучшим среди одиночных тэггеров оказался Unigram, поэтому все
лучшие комбинации были с ним в составе. Добавление в качестве
последнего тэггера дефолтный даёт прирост всем комбинациям, что
довольно логично, ведь мы вместо None на незнакомых словах ставим
тэг существительного, и иногда попадаем.



Тэггер, который является аналогом Unigram:

In [ ]:
from collections import defaultdict

In [ ]:
class MyUniTagger(nltk.tag.SequentialBackoffTagger):
    def __init__(self, train, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # В ключах defaultdict будут тэги текущего слова
        self.dictionary = {}
        
        for sent in train:
            for word, tag in sent:
                if word is None:
                    continue
                    
                word = word.lower()
                if word not in self.dictionary:
                    self.dictionary[word] = defaultdict(int)
                self.dictionary[word][tag] += 1
            
    
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word is None:
            return None
        word = word.lower()
        
        # Возвращаем наиболее популярный тэг текущего слова
        if word in self.dictionary:
            return sorted(self.dictionary[word].items(), key=lambda x: x[1], reverse=True)[0][0]
        
        return None

Оценка тэггера:

In [ ]:
my_unitagger = MyUniTagger(fdata_train)

In [ ]:
my_unitagger.accuracy(fdata_test)

0.8829611302819194

Добавление в таблицу метрик новые данные

In [ ]:
comparison_mytaggers = pd.concat((comparison, pd.DataFrame(round(my_unitagger.accuracy(fdata_test), 3), 
                                                       index=['MyUniTagger'], columns=['Score'])), axis=0)

comparison_mytaggers = pd.concat((comparison_mytaggers, pd.DataFrame(round(MyUniTagger(fdata_train, backoff=default_tagger)\
                                                             .accuracy(fdata_test), 3), 
                                                       index=['MyUniTagger + Default'], columns=['Score'])), axis=0)

comparison_mytaggers.sort_values('Score', ascending=False, inplace=True)
comparison_mytaggers

,Score
Trigram + Unigram + Default,0.912
Trigram + Bigram + Unigram + Default,0.912
Bigram + Unigram + Default,0.912
MyUniTagger + Default,0.908
Unigram + Default,0.906
Bigram + Unigram,0.884
Trigram + Unigram,0.883
Trigram + Bigram + Unigram,0.883
MyUniTagger,0.883
Unigram,0.878


Наш тэггер оказался чуть лучше Unigram тэггера nltk, а его комбинация
с дефолтным вошла в топ 5 результатов.

Теперь обучим логистическую регрессию на мультиклассовую
классификацию, чтобы предсказывать тэги. 


Сделаем предобработку:

In [ ]:
train_tok = []
y_train = []
for sent in fdata_train[:]:
    for tok in sent:
        if (tok[0] is None) or (tok[1] is None):
            continue
        train_tok.append(tok[0])
        y_train.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
y_test = []
for sent in fdata_test[:]:
    for tok in sent:
        if (tok[0] is None) or (tok[1] is None):
            continue
        test_tok.append(tok[0])
        y_test.append('NO_TAG' if tok[1] is None else tok[1])

In [ ]:
#  Распределение классов
pd.Series(y_test).value_counts(normalize=True)

NOUN     0.236348
PUNCT    0.190354
VERB     0.111593
ADJ      0.098503
ADP      0.089464
ADV      0.050761
PRON     0.048550
CCONJ    0.036993
PROPN    0.035695
PART     0.033426
DET      0.027817
SCONJ    0.018686
NUM      0.011309
AUX      0.009066
X        0.000874
SYM      0.000404
INTJ     0.000157
dtype: float64

Самый
популярный класс имеет долю 23%, а это далеко от метрики лучших тэггеров.




Сначала применим векторизацию по символам:

## TfidfVectorizer

In [ ]:
vectorizer_char = TfidfVectorizer(ngram_range=(1, 6), analyzer='char')

In [ ]:
X_train = vectorizer_char.fit_transform(train_tok)
X_test = vectorizer_char.transform(test_tok)

In [ ]:
X_train.shape, X_test.shape

((1204640, 304810), (153325, 304810))

In [ ]:
%%time
lr_char = LogisticRegression(random_state=RANDOM_STATE, solver='sag')
lr_char.fit(X_train, y_train)

CPU times: user 1min 30s, sys: 80.6 ms, total: 1min 30s
Wall time: 1min 30s


LogisticRegression(random_state=42, solver='sag')

In [ ]:
pred = lr_char.predict(X_test)

In [ ]:
lr_char_score_tfdf = accuracy_score(y_test, pred)
lr_char_score_tfdf

0.946518832545247

## HashingVectorizer

In [ ]:
vectorizer_char = HashingVectorizer(ngram_range=(1, 6), analyzer='char')

In [ ]:
X_train = vectorizer_char.fit_transform(train_tok)
X_test = vectorizer_char.transform(test_tok)

In [ ]:
X_train.shape, X_test.shape

((1204640, 1048576), (153325, 1048576))

In [ ]:
%%time
lr_char = LogisticRegression(random_state=RANDOM_STATE, solver='sag')
lr_char.fit(X_train, y_train)

CPU times: user 2min 7s, sys: 337 ms, total: 2min 7s
Wall time: 2min 6s


LogisticRegression(random_state=42, solver='sag')

In [ ]:
pred = lr_char.predict(X_test)

In [ ]:
lr_char_score_hv = accuracy_score(y_test, pred)
lr_char_score_hv

0.9425403554541008

## CountVectorizer

In [ ]:
vectorizer_char = CountVectorizer(ngram_range=(1, 6), analyzer='char')

In [ ]:
X_train = vectorizer_char.fit_transform(train_tok)
X_test = vectorizer_char.transform(test_tok)

In [ ]:
X_train.shape, X_test.shape

((1204640, 304810), (153325, 304810))

In [ ]:
%%time
lr_char = LogisticRegression(random_state=RANDOM_STATE, solver='sag')
lr_char.fit(X_train, y_train)

CPU times: user 7min 50s, sys: 475 ms, total: 7min 50s
Wall time: 7min 48s


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(random_state=42, solver='sag')

In [ ]:
pred = lr_char.predict(X_test)

In [ ]:
lr_char_score_cv = accuracy_score(y_test, pred)
lr_char_score_cv

0.9509669003750204

Значительынй прирост в метрике. Лучший результат показал CountVectorizer, но модель обучалась значительно дольше.

Применим векторизацию по словам:

## TfidfVectorizer

In [ ]:
vectorizer_word = TfidfVectorizer(ngram_range=(1, 2), analyzer='word')

In [ ]:
X_train = vectorizer_word.fit_transform(train_tok)
X_test = vectorizer_word.transform(test_tok)

In [ ]:
%%time
lr_word = LogisticRegression(random_state=RANDOM_STATE, solver='sag')
lr_word.fit(X_train, y_train)

CPU times: user 52.9 s, sys: 290 ms, total: 53.2 s
Wall time: 52.9 s


LogisticRegression(random_state=42, solver='sag')

In [ ]:
pred = lr_word.predict(X_test)

In [ ]:
lr_word_score_tfdf = accuracy_score(y_test, pred)
lr_word_score_tfdf

0.7632806130767976

## HashingVectorizer

In [ ]:
vectorizer_word = HashingVectorizer(ngram_range=(1, 2), analyzer='word')

In [ ]:
X_train = vectorizer_word.fit_transform(train_tok)
X_test = vectorizer_word.transform(test_tok)

In [ ]:
%%time
lr_word = LogisticRegression(random_state=RANDOM_STATE, solver='sag')
lr_word.fit(X_train, y_train)

CPU times: user 53.4 s, sys: 437 ms, total: 53.8 s
Wall time: 53.7 s


LogisticRegression(random_state=42, solver='sag')

In [ ]:
pred = lr_word.predict(X_test)

In [ ]:
lr_word_score_hv = accuracy_score(y_test, pred)
lr_word_score_hv

0.7609326593836622

## CountVectorizer

In [ ]:
vectorizer_word = CountVectorizer(ngram_range=(1, 2), analyzer='word')

In [ ]:
X_train = vectorizer_word.fit_transform(train_tok)
X_test = vectorizer_word.transform(test_tok)

In [ ]:
%%time
lr_word = LogisticRegression(random_state=RANDOM_STATE, solver='sag')
lr_word.fit(X_train, y_train)

CPU times: user 1min 29s, sys: 99.9 ms, total: 1min 29s
Wall time: 1min 29s


LogisticRegression(random_state=42, solver='sag')

In [ ]:
pred = lr_word.predict(X_test)

In [ ]:
lr_word_score_cv = accuracy_score(y_test, pred)
lr_word_score_cv

0.7632480026088374

Tfidf показал чуть лучше результат чем CountVectorizer.
HashingVectorizer хуже остальных.


Внесём все данные в таблицу:

In [ ]:
comparison_mytaggers_vectorizers = pd.concat((comparison_mytaggers, 
                                              pd.DataFrame(round(lr_char_score_tfdf, 3), index=['Char Tfidf + Logistic Regression'], 
                                                                                    columns=['Score'])), axis=0)
comparison_mytaggers_vectorizers = pd.concat((comparison_mytaggers_vectorizers, 
                                              pd.DataFrame(round(lr_char_score_hv, 3), index=['Char HashingVectorizer + Logistic Regression'], 
                                                                                    columns=['Score'])), axis=0)
comparison_mytaggers_vectorizers = pd.concat((comparison_mytaggers_vectorizers, 
                                              pd.DataFrame(round(lr_char_score_cv, 3), index=['Char CountVectorizer + Logistic Regression'], 
                                                                                    columns=['Score'])), axis=0)
comparison_mytaggers_vectorizers = pd.concat((comparison_mytaggers_vectorizers, 
                                              pd.DataFrame(round(lr_word_score_tfdt, 3), index=['Word Tfidf + Logistic Regression'], 
                                                                                    columns=['Score'])), axis=0)
comparison_mytaggers_vectorizers = pd.concat((comparison_mytaggers_vectorizers, 
                                              pd.DataFrame(round(lr_word_score_hv, 3), index=['Word HashingVectorizer + Logistic Regression'], 
                                                                                    columns=['Score'])), axis=0)
comparison_mytaggers_vectorizers = pd.concat((comparison_mytaggers_vectorizers, 
                                              pd.DataFrame(round(lr_word_score_cv, 3), index=['Word CountVectorizer + Logistic Regression'], 
                                                                                    columns=['Score'])), axis=0)

comparison_mytaggers_vectorizers.sort_values('Score', ascending=False, inplace=True)
comparison_mytaggers_vectorizers

,Score
Char CountVectorizer + Logistic Regression,0.951
Char Tfidf + Logistic Regression,0.947
Char HashingVectorizer + Logistic Regression,0.943
Trigram + Unigram + Default,0.912
Bigram + Unigram + Default,0.912
Trigram + Bigram + Unigram + Default,0.912
MyUniTagger + Default,0.908
Unigram + Default,0.906
Bigram + Unigram,0.884
Trigram + Unigram,0.883


С большим отрывом лидируют логистическая регрессия с Tfidf, CountVectorizer и HashingVectorizer векторизацией
по символам. Лучший результат CountVectorizer. Векторизация по словам оказалась значительно хуже, и это объяснимо:
ведь о части речи может говорить какая-то часть слова, например, окончание, а не
всё слово целиком.